## Cine GPT

Pequeno assistente virtual (Cine GPT) para um indicador de filmes utilizando uma base como referência (RAG).

Foi utilizada a base de filmes do IMDB top 1000, restrita a 50 filmes para economizar em tokens.

Utilizando SentenceTransformers, convertemos o filme, genero e sinopse em embeddings.
Utilizando FAISS encontramos filmes para uma pergunta do usuário.

Utilizando um modelo de linguagem generativo pela API do hugging Face, o Zephyr 7b para as respostas.
O modelo é leve e performa bem para tarefas simples. Além disso o prompt inicial
é enriquecido com informações relevantes sobre os filmes que
foram dados como resultados dos embeddings.

Há um laço de interação contínua do usuário com a plataforma, por razão de limitação de memória o modelo não consegue dispor de uma 
memória sobre as consultas, mas é algo plenamente possível com essa arquitetura uma vez que se insira o contexto nos prompts seguintes.

Ao utilizar o assistente, pergunte continuamente e quando quiser encerrar digite 'sair'.


Bibliotecas utilizadas:
- huggingface_hub
- sentence-transformers
- pandas
- faiss-cpu

In [2]:
#Bibliotecas utilizadas

import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from huggingface_hub import InferenceClient

TOKEN_HUGGING_FACE = 'seu-token'
#Criação de base de dados
df = pd.read_csv("IMDB top 1000.csv") #Arquivo CSV com filmes indexados, dataset clássico de estudo. Apenas 50 filmes estão catalogados.
df = df[['Title', 'Description', 'Genre']].dropna() #Restrição para colunas de interesse
df['full_text'] = df['Title'] + ". " + df['Genre'] + ". " + df['Description'] #Criação da coluna full text para criar os embeddings.

#Criação de embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2") #modelo gerador de embeddings
embeddings = embedder.encode(df['full_text'].tolist()) #geração de embeddings.

# Criação dos vetores de similaridade (FAISS)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

# Cliente Hugging Face Client Call
client = InferenceClient(
    model="HuggingFaceH4/zephyr-7b-beta",  # mais leve que LLaMA e funcional grátis
    token=
)

# A função buscar filmes utiliza uma busca diretamente no modelo de embeddings e o valor de k é a quantidade que o modelo deve indicar.
# Convencionamos em 3, mas podem ser mais ou menos.
def buscar_filmes(pergunta, k=3):
    query_emb = embedder.encode([pergunta])
    _, idx = index.search(np.array(query_emb), k)
    return df.iloc[idx[0]]

# Prompt inicial que pega uma parte dos resultados de filmes como contexto para detalhar no
# prompt que ajuda o modelo a ser mais acurado em suas respostas.
def montar_prompt(pergunta, filmes):
    intro = (
        "Você é um assistente de um cinema. Responda em português do Brasil com base nos filmes abaixo, mesmo que contenha dados em outro idioma, traduza:\n\n"
    )
    contexto = "\n".join(
        [f"{row['Title']} ({row['Genre']}): {row['Description'][:200]}..."
         for _, row in filmes.iterrows()]
    )
    return f"{intro}{contexto}\n\nVocê: {pergunta}\nCine GPT:"

#Mensagem de boas vindas.
print("Bem vindo ao Cine GPT! Digite sua pergunta ou 'sair' para encerrar.")

#Laço de repetição da conversa entre o usuário e o LLM
# O fluxo é o seguinte: Pergunta do usuário -> Busca na base de filmes com embeddings -> ###
# -> montagem de prompt para resposta -> análise de LLM -> resposta final para usuário. 
while True:
    entrada = input("Você: ") #entrada de código
    if entrada.lower() == "sair": #em caso de saída do modelo uma mensagem de despedida é feita e o código é encerrado. 
        print('Cine GPT: Obrigado por utilizar nossos serviços. Até mais!')
        break

    filmes = buscar_filmes(entrada) #análise da entrada na base de filmes
    prompt = montar_prompt(entrada, filmes) # criação da resposta com base nos filmes destacados.

    resposta = client.text_generation(
        prompt=prompt,
        max_new_tokens=300,
        temperature=0.7,
        do_sample=True,
        stop=["Você:", "Cine GPT:"]
    ) #Resposta gerada, 300 tokens para ser uma resposta relativamente curta e com temperatura controlada para uma boa desenvoltura, 
    # mas sem alucinar.

    print("Cine GPT:", resposta.strip(), "\n") #print da resposta e volta para o while True.

Bem vindo ao Cine GPT! Digite sua pergunta ou 'sair' para encerrar.


Você:  Me indique um bom filme de ação


Cine GPT: Para você, eu recomendo o filme "Joker" de 2019. Dirigido por Todd Phillips, com Joaquin Phoenix no papel principal, é um filme de drama, com elementos de comédia, terror e ação. Se você gosta de histórias complexas e personagens complexos, este filme é para você.

Você: 



Você:  Me dê informações sobre um bom filme de aventura


Cine GPT: Um filme de aventura de grande qualidade é "Indiana Jones e o Templo da Perdição" (2008), de Steven Spielberg e George Lucas. A aventura retorna ao personagem arqueólogo Indiana Jones (Harrison Ford) quando ele é recrutado pela Universidade de Yale para recuperar o Cálice da Santíssima Trindade, um antigo e místico artefato que pode ressuscitar um exército perdido da Cruzada. A ação se desenrola em locais exóticos, com um ritmo rápido e maravilhosos efeitos especiais. É um filme de ação, com humor e aventura, destinado a todos os públicos. 



Você:  sair


Cine GPT: Obrigado por utilizar nossos serviços. Até mais!
